In [1]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess

from osgeo import gdal
from osgeo import gdal_array
from osgeo_utils import gdal_calc

import os
from typing import Tuple

In [2]:
import os, datetime
from typing import Tuple


from osgeo import gdal


def getBand(band_name: str) -> int | str | None:
    ret = band_name.split(".")[0]
    if "QA_PIXEL" in band_name: return "QA"
    if "NDSI" in band_name: return "NDSI"
    ret = ret[len(ret) - 2:len(ret)]
    if ret[1].isnumeric() and ret[0] == "B": return int(ret[1])
    return None


def getNameNoBand(band_name: str) -> str:
    return band_name[0:40]


def isTiffImage(file: str):
    return file[len(file) - 4:len(file)].upper() == ".TIF" or file[len(file) - 5:len(file)].upper() == ".TIFF"


def doRecursiveSearch(folder: str, filter_function=None) -> list[tuple[str, str]]:
    folders = os.walk(folder)
    ret = []
    for folder in folders:
        files = os.listdir(folder[0])
        for file in files:
            if filter_function == None or filter_function(file):
                ret.append((file, folder[0]))
    return ret


def findBandForImage(img: tuple[str, str], band_index: int | str, folder = None) -> tuple[str, str] | None:
    if folder is None: folder = img[1]
    files = doRecursiveSearch(folder, isTiffImage)
    name = getNameNoBand(img[0])
    for f in files:
        band = getBand(f[0])
        # print(f[0] + " -> " + str(band))
        if name == getNameNoBand(f[0]) and band_index == band: return f
    return None


def loadBand(img: tuple[str, str]) -> gdal.Dataset | None:
    return gdal.Open(imgPath(img))


def imgPath(img: tuple[str,str]):
    return img[1] + "\\" + img[0]

def isFromTimePeriod(img_name: str, min: datetime.date, max: datetime.date) -> bool:
    date = datetime.date(int(img_name[17:21]), int(img_name[21:23]), int(img_name[23:25]))
    return isTiffImage(img_name) and (min <= date <= max)


In [3]:
import subprocess

from osgeo_utils import gdal_calc

def calcNDSI(thermal, green, out_file: str):
    gdal_calc.Calc("(A.astype(numpy.float64)-B)/numpy.maximum(1,A.astype(numpy.float64)+B)",
                   A=imgPath(thermal),
                   B=imgPath(green),
                   outfile=out_file,
                   NoDataValue=-10.0,
                   overwrite=True,
                   quiet=True,
                   type="Float64"
                   )


def applyCloudMask(ndsi, quality, out_file):
    gdal_calc.Calc("A*(1-((B==22280)+(B==24088)+(B==24216)+(B==24344)+(B==24472)+(B==55052)))",
                   A=imgPath(ndsi),
                   B=imgPath(quality),
                   outfile=out_file,
                   NoDataValue=-0.0,
                   overwrite=True,
                   quiet=True
                   )


def cropToShapefile(img_from, img_to, shape):
    print("Cropping " + img_from + " -> (" + shape + ") -> " + img_to)
    subprocess.call(['gdalwarp', img_from, img_to, '-cutline', shape, '-crop_to_cutline', '-q'], stderr=None)


def genMosaic(imgs: list[tuple[str, str]], img_to: str):
    args = []
    for i in imgs: args.append(i[1] + "\\" + i[0])
    # gdal_merge.main(['','', '-o', img_to] + args)
    subprocess.call(['gdalwarp','-r','average'] + args + [img_to], stderr=None)


def mosaicAndShape(imgs: list[tuple[str, str]], img_to: str, shapefile: str):
    args = []
    print("generating mosaic " + img_to)
    for i in imgs: args.append(i[1] + "\\" + i[0])
    subprocess.call(['gdalwarp','-cutline',shapefile,'-crop_to_cutline','-r','average','-q'] + args + [img_to], stderr=None)
    print(args)


In [11]:

ee.Authenticate()
ee.Initialize(project='ee-juliocesarborgesdeoliv-neve')

print("Olá mundo")

study_area_broad = ee.Geometry.Rectangle([-104.05, 49.05, -96.55, 45.05])

landsat_8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(study_area_broad).filterDate('1980-01-01','2024-01-01').sort("CLOUD_COVER_LAND").limit(10)

for img in landsat_8.getInfo()["features"]:
    print("Found img " + img["id"])
    print(img["properties"]["CLOUD_COVER_LAND"])

exit(True)

# for year in range(1980, 2024):
#   print("Year " + str(year))
#   imgs = []
#   for month in range(1, 13, 4):
#     print("Range {:02d} - {:02d}".format(month, month + 3))

# landsat_7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
# landsat_5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
# landsat_4 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")




Olá mundo
Found img LANDSAT/LC08/C02/T1_L2/LC08_029026_20160501
0
Found img LANDSAT/LC08/C02/T1_L2/LC08_029026_20170909
0
Found img LANDSAT/LC08/C02/T1_L2/LC08_029026_20200512
0
